#### PIP installations

In [ ]:
# !pip install spacy
# !pip install textstat
# !python -m spacy download en_core_web_sm
# !pip install langchain_mistralai

#### LLM config

In [ ]:

import requests
import json
import base64

def encode_image(image_path):
        with open(image_path, "rb") as img_file:
                return base64.b64encode(img_file.read()).decode("utf-8")
def query_llm_rest_pix(mlist,image=None):
        
        url = 'http://172.19.104.12:9104/v1/chat/completions'
        headers = {
            'Content-Type': 'application/json',
            'Authorization': 'Bearer token'
        }


        data = {
            "model": "mistralai/Pixtral-12B-2409"
        }
        data.update({"messages": mlist})

        response = requests.post(url, headers=headers, data=json.dumps(data))

        if response.status_code == 400:
            return "I'm unsure about the answer. Please provide more context or ask a different question."
        else:
            return response.json()["choices"][0]['message']['content']




#### sythetic dataset creation

##### Dataset Creation Approach

    There are lot of datasets available on the internet for varies llm evaluation tasks. 

    Dataset benchmarks like 
```
    arc-challenge
    mbpp
    winogrande
    mtbench
    grade-school-math
    hellaswag
    mmlu
```
    However, categorizing, downloading and preprocessing those datasets was taking more time for me than anticipated initially. Because of this reason, I will be taking a sythetic dataset creation approach to generate the data.

##### I will be using the LLMs to generate the data for the modules defined in the assignment.
```
    Document Extraction
    Quantitative Analysis
    Report Generation
    Interactive QA Chatbot
    Multi Document Summarization
```



##### Define module definition

In [ ]:
modules = {
    "t1":"Document Content Extraction: Reading documents in different formats and extracting information from it, which could be in tables, graphs, etc. Make sure all quantitative data is extracted correctly with the right context.",
    "t2": "Quantitative Analysis: To streamline and automate the process of performing advanced calculations based on given parameters. If required, the LLM needs to choose the correct parameters like assumptions, margin of error, etc.",
    "t3": "Report Generation: Creating a report following a pre-set format, length, and style, including text, charts, and illustrations. This can be calibrated based on requirements and data sources.",
    "t4":"Interactive QA Chatbot: A question-and-answer chatbot to go with the final report so that if there are any queries, they can be answered based on all the sources used for creating the report.",
    "t5":"Multi-document Summarization: Summarizing data from multiple documents, giving importance to the most recent information or specified context."
}

maping = {
"t1":"document_extraction",
"t2":"quantitative_analysis",
"t3":"report_generation",
"t4":"interactive_qa_chatbot",
"t5":"multi_document_summarization"
}


##### Prompt Iteration 

I refined and enhanced the prompt to optimize prompt generation. Below is the final version that was used. I'm using pixtral local model here. Using bigger model will certainly yield better results.

In [ ]:
prompt = f"""
    You are a prompt generation expert who can generate prompts for variety of tasks. You have been given the following description of task. 
    The prompt you generate should be of varying length and complexity. Give the response stricty in python dictionary format.
    {{"prompt":"","complexity":"","data_type":""}}

    data_type can include following types["Text", "Numerical Data", "Tables", "Graphs & Charts", "Equations & Formulas", "Dates & Timestamps"]
    complexity can include following types["Simple", "Medium", "Complex"]
    
    Reponse only in text. Dont include markdown.
    Generate 50 prompts for each of the following tasks:
        Document Content Extraction: Reading documents in different formats and extracting information from it, which could be in tables, graphs, etc. Make sure all quantitative data is extracted correctly with the right context.

    """

In [ ]:
import json

num_prompts = 50

prompt_dict = []

# loop through modules
for name, module in modules.items():
    prompt = f"""
    You are a prompt generation expert who can generate prompts for variety of tasks. You have been given the following description of task. 
    The prompt you generate should be of varying length and complexity. Give the response stricty in python dictionary format.
    {{"prompt":"","complexity":"","data_type":""}}

    data_type can include following types["Text", "Numerical Data", "Tables", "Graphs & Charts", "Equations & Formulas", "Dates & Timestamps"]
    complexity can include following types["Simple", "Medium", "Complex"]
    
    Reponse only in text. Dont include markdown.
    Generate {num_prompts} prompts for each of the following tasks:
        {module}

    """
    mlist =[ {       
            "role": "system",
            "content": [
                {"type": "text", "text": "You are a prompt generation expert who can generate prompts for variety of tasks."}
            ]
        },
        {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt}
                    ]
        }
    ]
    res = query_llm_rest_pix(mlist)
    print(res.content)
    prompt_dict.append({"module":name,"prompts":res})
    
with open("v1.json", "w") as f:
    json.dump(prompt_dict, f, indent=4)


### Feature Explanations for LLM Prompt Classification

- **`num_tokens`** – Helps determine prompt length, affecting processing time and model performance.  
- **`num_sentences`** – Indicates prompt complexity and structure.  
- **`lexical_diversity`** – Measures vocabulary richness, impacting generalization.  
- **`readability`** – Assesses how easy or difficult the prompt is to understand.  
- **`noun_ratio`** – Identifies the amount of entity-related information.  
- **`verb_ratio`** – Shows action-oriented nature of the prompt.  
- **`adjective_ratio`** – Captures descriptive complexity in the prompt.  
- **`num_named_entities`** – Detects presence of key entities, aiding context understanding.  
- **`contains_table`** – Flags structured data presence, affecting LLM parsing needs.  
- **`contains_list`** – Identifies enumerated elements that may require special processing.  
- **`keyword_density`** – Measures topic emphasis and focus within the prompt.  
- **`redundancy_score`** – Helps detect excessive repetition, which may confuse LLMs.  
- **`compression_ratio`** – Evaluates information density, useful for summarization tasks.  
- **`contains_numbers`** – Determines presence of quantitative data, influencing numerical reasoning.  
- **`contains_chain_of_thought`** – Checks for step-by-step reasoning, important for logical tasks.  
- **`contains_output_constraints`** – Identifies explicit formatting constraints in expected responses.  
- **`is_multi_turn`** – Detects conversational prompts requiring memory or context retention.  


In [ ]:
import spacy
import textstat
import re
from collections import Counter

# Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")

def extract_prompt_features(prompt):
    doc = nlp(prompt)
    
    #  Linguistic Features
    num_tokens = len(doc)
    num_sentences = len(list(doc.sents))
    unique_words = len(set([token.text.lower() for token in doc]))
    lexical_diversity = unique_words / num_tokens if num_tokens > 0 else 0
    readability = textstat.flesch_kincaid_grade(prompt)
    
    # Part-of-Speech (POS) Distribution
    pos_counts = Counter([token.pos_ for token in doc])
    noun_ratio = pos_counts.get("NOUN", 0) / num_tokens
    verb_ratio = pos_counts.get("VERB", 0) / num_tokens
    adjective_ratio = pos_counts.get("ADJ", 0) / num_tokens

    #  Structural Features
    num_named_entities = len(doc.ents)
    contains_table = bool(re.search(r"(\||\+\-+)|Table \d+", prompt))  # Detect tables
    contains_list = bool(re.search(r"(\d+\.)|(- )|(\* )", prompt))  # Detect bullet points or numbered lists

    # Content-Specific Features
    keyword_density = {word: prompt.lower().count(word) for word in ["summary", "analyze", "data", "report"]}
    redundancy_score = len(re.findall(r"(\b\w+\b).*\1", prompt))  # Count repeated words
    compression_ratio = len(prompt) / num_tokens if num_tokens > 0 else 0

    #  Task-Specific Complexity Features
    contains_numbers = bool(re.search(r"\d+", prompt))  # Presence of numerical data
    contains_chain_of_thought = bool(re.search(r"step-by-step|explain your reasoning", prompt.lower()))
    contains_output_constraints = bool(re.search(r"limit to|output in|return a json", prompt.lower()))
    is_multi_turn = "previous response" in prompt.lower() or "as mentioned before" in prompt.lower()

    return {
        "num_tokens": num_tokens,
        "num_sentences": num_sentences,
        "lexical_diversity": lexical_diversity,
        "readability": readability,
        "noun_ratio": noun_ratio,
        "verb_ratio": verb_ratio,
        "adjective_ratio": adjective_ratio,
        "num_named_entities": num_named_entities,
        "contains_table": contains_table,
        "contains_list": contains_list,
        "redundancy_score": redundancy_score,
        "compression_ratio": compression_ratio,
        "contains_numbers": contains_numbers,
        "contains_chain_of_thought": contains_chain_of_thought,
        "contains_output_constraints": contains_output_constraints,
        "is_multi_turn": is_multi_turn
    }

# Example Prompt
prompt = """
You are an AI assistant. Summarize the research paper and provide key insights. 
Ensure the response is formatted as JSON and limited to 200 words. 
Step-by-step analysis is preferred. Example: "In this study, the authors examined..."
"""

# Extract Features
features = extract_prompt_features(prompt)
print(features)


##### Extract for all prompts

In [ ]:
i = 0
data = []
maping = {
"t1":"document_extraction",
"t2":"quantitative_analysis",
"t3":"report_generation",
"t4":"interactive_qa_chatbot",
"t5":"multi_document_summarization"
}

maping = {
"t1":"GPT",
"t2":"Claude",
"t3":"Mistral",
"t4":"llma",
"t5":"GPT"
}
for mod in prompt_dict:
    for prompt in mod["prompts"]:
        print(prompt)
        pt = prompt["prompt"]
        complx = prompt["complexity"]
        data_type = prompt["data_type"]
        inter = {"PID": i,"prompt_length":len(pt),  "complexity": complx, "data_type": data_type,"module": maping[mod["module"]]}
        
        inter.update(extract_prompt_features(pt))
        
        i += 1
        data.append(inter)


with open("v2.json", "w") as f:
    json.dump(data, f, indent=4)

#### csv

In [ ]:
import csv
csv_file = "input.csv"

# Write to CSV
with open(csv_file, mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=data[0].keys())
    writer.writeheader()  # Write column headers
    writer.writerows(data)  # Write rows

print(f"CSV file '{csv_file}' created successfully!")

#### Introduce some noise

In [ ]:
import pandas as pd
import numpy as np

# Load CSV file
df = pd.read_csv("input.csv")

# Define the target column
label_col = "module"  # Replace with your actual column name
new_col = "LLM"

# Get unique labels
labels = df[label_col].unique()

# Create a new column with the same values initially
df[new_col] = df[label_col]

# Modify 20% of each label group
for label in labels:
    label_indices = df[df[label_col] == label].index
    num_to_replace = int(0.2 * len(label_indices))  # 20% of each label

    if num_to_replace > 0:
        # Choose random indices to modify
        replace_indices = np.random.choice(label_indices, num_to_replace, replace=False)

        # Select different labels for replacement
        possible_labels = [l for l in labels if l != label]
        df.loc[replace_indices, new_col] = np.random.choice(possible_labels, num_to_replace)

# Save the modified CSV
df.to_csv("modified_file.csv", index=False)

print("CSV file updated with new column!")
